## 탐욕법 (Greedy Algorithm)
 - 알고리즘의 각 단계에서 그 순간에 최적이라고 생각되는 것을 선택
 - 탐욕법으로 최적해를 찾을 수 있는 문제: 현재의 선택이 마지막 해답의 최적성을 해치지 않을 때

    
### <font color = blue> **그리디 포인트** 


 - 손으로 직접 예제를 노트에 그려보며 어떻게 풀어나갈지 감을 잡는다
    - 예제가 만들어지는 과정을 천천히 직접 손으로 따라가며 진행
    - 진행되는바에 따른 프로세스를 명확하게 이해하고 규칙을 생각하여 코드로 구현할 생각 
    - 반복문 / 코드문 내부에 어떻게 어떠한 규칙이 작동할지를 예제를 통해서 잘 이해하고 작성
   
    
 - 풀면서 문제의 예시에 적용해 나간 것들을 코드로 옮김
    - 맨 처음 혹은 맨 끝에 경우에 있어서 또 다른 경우의 수를 생각해 주어야 하는 경우  
        -> 효율과 편의성 있는 코딩을 위해서 사용하지 않는 1을 양 끝에 넣을 수 있음
    - 문제를 규칙에 따라 순서대로 적용해 나가며 풀수도 있지만 이해하고 다르게 풀 수도 있음  
        -> 순서대로 경우의 수 카운트하는 대신 전체를 모두 세고 예외되는 것만 카운트하여 빼는 방식  
        -> 문제의 요구사항을 명확히 파악하고 더 나은 아이디어를 적용해서 푸는 것  
    
    
 

### 1. 체육복

#### 탐욕법 적용 가능성 확인
 - 빌려줄 학생들을 "정해진 순서"로 살펴야 하고, 이 "정해진 순서"에 따라 우선하여 빌려줄 방향을 정해야 함



#### 문제의 해결 - 방법(1)
 - (착안점) 학생의 수는 기껏해야 30!
 - 학생 수만큼 배열을 확보하고, 여기에 각자가 가지고 있는 체육복의 수를 기록한다.
   -> 번호 순서대로 "스캔"하면서 빌려줄 **관계**를 정한다.

#### 알고리즘 복잡도(1)
 - 여벌을 가져온 학생 처리: reserve의 길이에 비례
 - 체육복을 잃어버린 학생 처리: lost의 길이에 비례
 - 체육복을 빌려주기 처리: 전체 학생 수 (n)에 비례
   -> O(n)

In [ ]:
def solution(n, lost, reserve):
    # 풀이1    
    uniforms = [1] * (n + 2)
    for x in reserve:
        uniforms[x] += 1
    for x in lost:
        uniforms[x] -= 1
    for i in range(1, n+1):
        if uniforms[i - 1] == 0 and uniforms[i] == 2:
            uniforms[i - 1:i + 1] = [1, 1]
        elif uniforms[i] == 2 and uniforms[i + 1] == 0:
            uniforms[i : i + 2] = [1, 1]   
    
    return len([i for i in uniforms[1:n+1] if i > 0])

#### 문제의 해결 - 방법(2)
 - 만약 전체 학생 수가 매우 크다면?
 - 하지만 문제의 성질상 O(n)보다 낮은 복잡도 알고리즘은 어려울 듯?
 - 그런데 여벌의 체육복을 가져온 학생은 매우 적다면?  
    -> 여벌의 체육복을 가져온 학생들의 번호(reserve)를 정렬하고, 이것을 하나 하나 순서대로 살펴보면서 빌려줄 수 있는 다른 학생을 찾아서 처리한다! 
    
    -> 해시를 적용해서 상수 시간에 처리!

알고리즘의 복잡도
 - 여벌의 체육복을 가져온 학생들의 번호 (reverve)를 정렬
  -> O(klogk)
 - 체육복을 빌려줄 수 있는 학생을 찾아 처리
  -> O(k) x O(1)
 - 전체 알고리즘의 시간 복잡도
  -> O(klogk)

In [ ]:
def solution(n, lost, reserve):
    # 풀이2
    stolen_but_have = set(lost) & set(reserve)
    stolen_not_have = set(lost) - stolen_but_have
    have_reserve = set(reserve) - stolen_but_have
    
    for num in sorted(have_reserve):
        if (num - 1) in stolen_not_have:
            stolen_not_have.remove(num - 1)
        elif (num + 1) in stolen_not_have:
            stolen_not_have.remove(num + 1)
    
    return n - len(stolen_not_have)

### 2. 큰 수 만들기

1. 큰 수 만들기 - 원칙
 - 앞 자리에 큰 수가 오는 것이 전체를 크게 만든다
     - 따라서, 큰 것을 우선해서 골라 담고 싶다

2. 큰 수 만들기 - 방법
 - 앞 자리에서부터 하나씩 골라서 담되,
     -지금 담으려는 것보다 작은 것들은 도로 뺀다!
 - 단, 뺄 수 있는 수효에 도달할 때까지만  
이는 다음과 같다.
 - 큰 수가 앞 자리에, 작은 수가 뒷 자리에 놓이도록  
  (제약조건) 뺄 수 있는 개수 k 개


3. 알고리즘 설계 -> 구현
 - 주어진 숫자(number)로부터 하나씩 꺼내어 모으되
   - 이 때, 이미 모아둔 것 중 지금 등장한 것보다 작은 것들은 빼낸다.
 - 이것은 어디서 어떻게 살펴보아야?
 - 이렇게 모은 숫자들은 자리숫 맞추어 반환한다.
   - 아직 뺄 개수 (k)를 채우지 못한 경우
   - 자릿수는 어떻게 계산하는가?


4. 알고리즘의 복잡도
 - 가장 단순(무식?)한 방법은 어떤 것일까?
 - 우리가 설계한 알고리즘의 복잡도는?


5. 탐욕법 (Greedy Approach)
**탐욕법이 통할 수 있는 이유** - 앞 단계에서의 선택 (앞 자리에 큰수!)이 이후 단계에서의 동작에 의한 해(solution)의 최적성(optimality)에 영향을 주지 않음

In [1]:
def solution(number, k):
    collected = [ ]
    for i, num in enumerate(number):
        while len(collected) > 0 and collected[-1] < num and k > 0:
            collected.pop()
            k -= 1
        if k == 0:
            collected += list(number[i:])
            break
        collected.append(num)

    collected = collected[:-k] if k > 0 else collected
    answer = ''.join(collected)
    return answer